For many years there has been controversy regarding whether maternal employment has a detrimental or beneficial effect on children across a variety of dimensions.  The policy implications of such links range from considerations regarding when mothers participating in TANF (Temporary Assistance to Needy Families) should be forced to return to work after giving birth, to proposed changes in provisions under the Family and Medical Leave Act regarding length of leave and who is covered.  We will use data on a sample of women and children from the National Longitudinal Survey of Youth (data, codebook and description of survey in separate files on Blackboard in this section) to explore whether or not there appears to be a causal link between maternal employment and children’s cognitive development.

I will use propensity score matching to estimate the effect of maternal employment in the first year on children’s cognitive outcomes.  The sample has been restricted to exclude the children of mothers who did not work at all in the first three years of their children’s life in order to focus analyses on comparisons between the children of mothers who return to work in the first year after giving birth and those who return either in the 2nd or 3rd year of their child’s life.  Use math achievement scores from ages 5/6 “piatm56” as outcome variable.


First I will consider the effect of the treatment (maternal employment during first year of child’s life) on the treated (children whose mothers worked during the first year of their lives) where the treatment variable is called first.  Decide what variables should be considered confounding covariates in the subsequent analyses.  Then prioritize the confounders into three categories (most important to balance, somewhat less important to balance, least important to balance) with roughly the same number of variables in each category. When assessing model fit I'd like to evaluate balance on all of these variables, regardless of whether all of them end up in final propensity score model.  However this prioritization may help to decide how to evaluate trade-offs in balance across different model specifications.  

* List of all the confounding covariates:
Hispanic; black; white; momrace; female; hs; b_marr;  pr0; lths; momed; lnfinc_a0; brthwt; brorddum; preterm; afqt;  ltcoll; college; momage; age; rmomwk;

* Most important
Family income, Family poverty situation, weeks per-term, mother’s marital status, birthweight of the child;

* Less important:
Mom’s race and Education, mother’s age, first born or not

* Least important
Child’s age, mom’s score on the armed forces qualifying test, did respondent have a mom who worked when respondent was in high school, Female or not.


Some notes: I think the family income/poverty, mother’s marital status, and the health of the child are very important factors that are associated with both treatment (if go to work in the first year of giving birth) and the outcome (child’s PIAT math score at age 5-6). I think both birthweight and weeks of pre-term are related with child’s health. I think mom’s race and education, mom’s age, and if the child is first born or not, are also associated with the treatment and the outcome. But I think these factors could be less related to the treatment than family income and child’s health status. Other factors such as child’s age, mom’s score on the armed forces qualifying test, respondent’s mom employment status in high school, and if the child is female or not, are the least important confounding covariates. Even though I think they are associated with treatment and outcome, but I think they probably won’t affect the outcome significantly. 

## Search for a good propensity score model 

In [1]:
from pymatch.Matcher import Matcher
import pandas as pd
import numpy as np

In [2]:
df = pd.read_stata("/Users/YaoJunyan/Documents/NYU/Fall 2017/Causal Inference/Assignment 4/nlsy.hw4.forclass.dta")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3100 entries, 0 to 3099
Data columns (total 29 columns):
row_names    3100 non-null float32
hispanic     3100 non-null category
black        3100 non-null category
white        3100 non-null category
momrace      3100 non-null category
female       3100 non-null category
b_marr       3100 non-null category
pr0          3100 non-null category
lths         3100 non-null float32
hs           3100 non-null float32
ltcoll       3100 non-null float32
college      3100 non-null float32
momed        3100 non-null category
lnfinc_a0    3100 non-null float32
age          3100 non-null float32
momage       3100 non-null float32
afqt         3100 non-null float32
brthwt       3100 non-null float32
brorddum     3100 non-null category
preterm      3100 non-null float32
rmomwk       3100 non-null category
work         3100 non-null category
ppvt         3100 non-null float32
piatm56      3100 non-null float32
piatm78      3100 non-null float32
piatr56

### Make some adjustment on the variables

In [5]:
df.brorddum.value_counts()

first born    1677
2.0           1423
Name: brorddum, dtype: int64

In [6]:
df['brorddum2'] = np.where(df['brorddum']== 2, 0,1)

1 is for first born; 0 is for not first born


In [15]:
X= df[['hispanic', 'black' ,'white', 'momrace', 'female', 'hs' ,'b_marr', 'pr0', 'lths', 'momed', 'lnfinc_a0', 'brthwt', 'brorddum2', 'preterm', 'afqt', 'ltcoll', 'college', 'momage', 'age', 'rmomwk']]
Y= df['piatm56']
D= df['first']

In [29]:
df2= X= df[['first','hispanic', 'black' ,'white', 'momrace', 'female', 'hs' ,'b_marr', 'pr0', 'lths', 'momed', 'lnfinc_a0', 'brthwt', 'brorddum2', 'preterm', 'afqt', 'ltcoll', 'college', 'momage', 'age', 'rmomwk']]


test = df2[df2['first']==1]
control = df2[df2['first']==0]

In [30]:
m = Matcher(test, control, yvar="first", exclude=[])

Formula:
first ~ hispanic+black+white+momrace+female+hs+b_marr+pr0+lths+momed+lnfinc_a0+brthwt+brorddum2+preterm+afqt+ltcoll+college+momage+age+rmomwk
n majority: 2286
n minority: 814


In [31]:
np.random.seed(20170925)
m.fit_scores(balance=True, nmodels=100)

Fitting Models on Balanced Samples: 100\100
Average Accuracy: 64.68%


# Codebook

The NLSY (National Longitudinal Survey of Youth) has been collecting information on the same cohort of men and women since 1979 when they were teenagers.  Data was collected every year until 1992 and has been collected every two years since.  The sampling was done in such a way as to be nationally representative of that age cohort at the time the study was started.  The children of women in the study have been surveyed and tested every two years since 1986 as well.  The sample you have access to represents a selection of the variables for a (nonrandom) subset of these mothers and children.  

## description of variables

### hispanic, black, white
are dummy variables for mother's ethnicity
### momrace (1=hispanic, 2=black, 3=white/other)
### female
(1=female, 0=male)
### b_marr
mom married at birth? (1=yes)
### pr0
dummy for whether the household was in poverty in the year
before the child was born (1=yes)
### momed
mom's education level when she gave birth
(1= less than hs, 2=hs grad, 3=some college, 4=finished college)
#lths, hs, ltcoll, college are the corresponding dummies

### lnfinc_a0
logged family income, scaled by the poverty rate, in the year before the child was born

### age
child's age (in months) on 1/1/1990

### momage
age of mom when she gave birth (in years)

### afqt
mom's score on the armed forces qualifying test (an iq test of sorts)

### brthwt
birthweight of child in oz

### brorddum
1 if child first born, 0 otherwise

### preterm
weeks pre-term (note that if born late this could be negative)
all the ihdp kids were preterm

### rmomwk
did respondent (mother of child) have a mom who worked when 
respondent was in high school? (1=yes)

### work
1=mom did not work in 1st 3 years of child's life
2=mom did not work in 1st year of child's life but did work sometime
  in years 2 or 3
3=mom worked part-time in child's first year of life
4=mom worked full-time in child's first year of life

### ppvt
score on the Peabody Picture Vocab. Test (Revised) at 36 months
(a form of IQ test)

### piatm56
score on the PIAT math component at age 5 or 6
(a form of IQ test)

### piatr56
score on the PIAT reading component at age 5 or 6
(a form of IQ test)

### piatm78
score on the PIAT math component at age 7 or 8
(a form of IQ test)

### piatr78
score on the PIAT reading component at age 7 or 8
(a form of IQ test)
